# 1. Instalación de dependencias e importación de paquetes

In [142]:
# Install dependencies
!pip install pandas
!pip install numpy
!pip install -U scikit-learn
!pip install codecarbon

In [143]:
# Import packages
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn.svm import LinearSVC
from codecarbon import EmissionsTracker

# 2. Preprocesamiento del *dataset*: [Room Occupancy Estimation Data Set](https://www.kaggle.com/ananthr1/room-occupancy-estimation-data-set)

In [144]:
# Load data as dataframe
df = pd.read_csv('Occupancy_Estimation.csv')
df.shape

(10129, 19)

In [145]:
df.head()

,Date,Time,S1_Temp,S2_Temp,S3_Temp,S4_Temp,S1_Light,S2_Light,S3_Light,S4_Light,S1_Sound,S2_Sound,S3_Sound,S4_Sound,S5_CO2,S5_CO2_Slope,S6_PIR,S7_PIR,Room_Occupancy_Count
0,2017/12/22,10:49:41,24.94,24.75,24.56,25.38,121,34,53,40,0.08,0.19,0.06,0.06,390,0.769231,0,0,1
1,2017/12/22,10:50:12,24.94,24.75,24.56,25.44,121,33,53,40,0.93,0.05,0.06,0.06,390,0.646154,0,0,1
2,2017/12/22,10:50:42,25.00,24.75,24.50,25.44,121,34,53,40,0.43,0.11,0.08,0.06,390,0.519231,0,0,1
3,2017/12/22,10:51:13,25.00,24.75,24.56,25.44,121,34,53,40,0.41,0.10,0.10,0.09,390,0.388462,0,0,1
4,2017/12/22,10:51:44,25.00,24.75,24.56,25.44,121,34,54,40,0.18,0.06,0.06,0.06,390,0.253846,0,0,1


## Paso 1: Recodificación de la columna *RoomOccupancyCount*

En este caso, la columna que contiene la variable de salida (*RoomOccupancyCount*) cuenta el número de personas en la sala. Para simplificar, nuestro objetivo va a ser detectar si la sala está vacía o, en cambio, hay alguien (no el número exacto de personas).

Por lo tanto, se **recodificará cualquier valor mayor que 0 en esa columna como 1**, de tal forma que la variable de salida sea binaria:
- 0 -> ausencia.
- 1 -> presencia.

In [146]:
# Show "Room_Occupancy_Count" before processing
df["Room_Occupancy_Count"]

0        1
1        1
2        1
3        1
4        1
        ..
10124    0
10125    0
10126    0
10127    0
10128    0
Name: Room_Occupancy_Count, Length: 10129, dtype: int64

In [147]:
#Process "Room_Occupancy_Count"
df["Room_Occupancy_Count"] = np.where(df["Room_Occupancy_Count"] > 0, 1, 0)

# Show "Room_Occupancy_Count" after processing
df["Room_Occupancy_Count"]

0        1
1        1
2        1
3        1
4        1
        ..
10124    0
10125    0
10126    0
10127    0
10128    0
Name: Room_Occupancy_Count, Length: 10129, dtype: int32

## Paso 2: Eliminación de las columnas temporales *Date* y *Time*

Respecto a los datos ofrecidos por las columas *Date* y *Time*, vamos a filtrarlos y dejarlos fuera del proceso. Ya que, aunque los incluyeramos (por ejemplo, juntando ambas partes en una sola columna y convirtiéndolo a formato 'epoch', como long int), los valores de muestras sucesivas de esa columna estarían totalmente correlados entre sí y, como consecuencia, fastidiaríamos a la mayoría de algoritmos que vamos a emplear posteriormente.

Se debe tener en cuenta que no nos estamos olvidando de la información temporal para considerar como si cada valor muestreado (fila de la tabla) fuese independiente de las demás filas. Sabemos que eso no es así, pero estamos considerando que esa información temporal no nos ofrece valor añadido para predecir si la habitación está ocupada o vacía. En caso contrario, se tendrían que usar modelos bastante más complicados para considerar esa relación temporal que indica que, en realidad, las muestras de dos filas adyacentes son consecutivas en el tiempo.


In [148]:
# Filter Date and Time columns
df.drop(['Time'], axis=1, inplace=True)
df.drop(['Date'], axis=1, inplace=True)

# Show the result
df.head()

,S1_Temp,S2_Temp,S3_Temp,S4_Temp,S1_Light,S2_Light,S3_Light,S4_Light,S1_Sound,S2_Sound,S3_Sound,S4_Sound,S5_CO2,S5_CO2_Slope,S6_PIR,S7_PIR,Room_Occupancy_Count
0,24.94,24.75,24.56,25.38,121,34,53,40,0.08,0.19,0.06,0.06,390,0.769231,0,0,1
1,24.94,24.75,24.56,25.44,121,33,53,40,0.93,0.05,0.06,0.06,390,0.646154,0,0,1
2,25.00,24.75,24.50,25.44,121,34,53,40,0.43,0.11,0.08,0.06,390,0.519231,0,0,1
3,25.00,24.75,24.56,25.44,121,34,53,40,0.41,0.10,0.10,0.09,390,0.388462,0,0,1
4,25.00,24.75,24.56,25.44,121,34,54,40,0.18,0.06,0.06,0.06,390,0.253846,0,0,1


## Paso 3: División del *dataset* en *train* y *test*

In [149]:
# Split dataset into train and test
train_df, test_df = train_test_split(df) #75% training, 25% test
train_df.shape, test_df.shape

((7596, 17), (2533, 17))

In [150]:
train_df.head()

,S1_Temp,S2_Temp,S3_Temp,S4_Temp,S1_Light,S2_Light,S3_Light,S4_Light,S1_Sound,S2_Sound,S3_Sound,S4_Sound,S5_CO2,S5_CO2_Slope,S6_PIR,S7_PIR,Room_Occupancy_Count
3585,26.25,26.25,25.88,26.38,149,234,185,10,0.18,0.61,0.16,0.14,970,2.069231,0,1,1
6310,25.44,25.44,25.25,26.13,0,0,0,0,0.08,0.06,0.07,0.10,350,0.000000,0,0,0
7260,25.31,25.38,24.88,25.81,0,0,0,0,0.08,0.06,0.06,0.10,355,0.000000,0,0,0
1685,25.31,25.31,24.88,25.69,0,0,0,0,0.07,0.04,0.06,0.06,365,-0.084615,0,0,0
2933,25.69,25.38,25.19,25.94,123,38,70,50,0.12,0.08,0.11,0.09,380,0.288462,0,0,1


In [151]:
test_df.head()

,S1_Temp,S2_Temp,S3_Temp,S4_Temp,S1_Light,S2_Light,S3_Light,S4_Light,S1_Sound,S2_Sound,S3_Sound,S4_Sound,S5_CO2,S5_CO2_Slope,S6_PIR,S7_PIR,Room_Occupancy_Count
7166,25.38,25.38,24.94,25.88,0,0,0,0,0.08,0.06,0.06,0.08,355,0.000000,0,0,0
8252,25.88,26.06,25.75,25.94,3,3,19,12,0.88,0.38,0.15,0.08,740,2.707692,1,0,1
4347,25.44,25.44,24.94,25.69,0,0,0,0,0.06,0.04,0.06,0.06,365,-0.165385,0,0,0
8082,25.06,25.06,24.63,25.25,6,7,34,22,0.08,0.05,0.06,0.09,350,0.000000,0,0,0
4676,25.25,25.31,24.75,25.63,0,0,0,0,0.07,0.06,0.06,0.06,360,0.000000,0,0,0


## Paso 4: División del *dataset* de entrenamiento en *features* y *output*

In [152]:
# Get output column
train_output = train_df['Room_Occupancy_Count']
# Get features columns
train_features = train_df.copy()
train_features.drop(['Room_Occupancy_Count'], axis=1, inplace=True)

In [153]:
train_features.head()

,S1_Temp,S2_Temp,S3_Temp,S4_Temp,S1_Light,S2_Light,S3_Light,S4_Light,S1_Sound,S2_Sound,S3_Sound,S4_Sound,S5_CO2,S5_CO2_Slope,S6_PIR,S7_PIR
3585,26.25,26.25,25.88,26.38,149,234,185,10,0.18,0.61,0.16,0.14,970,2.069231,0,1
6310,25.44,25.44,25.25,26.13,0,0,0,0,0.08,0.06,0.07,0.10,350,0.000000,0,0
7260,25.31,25.38,24.88,25.81,0,0,0,0,0.08,0.06,0.06,0.10,355,0.000000,0,0
1685,25.31,25.31,24.88,25.69,0,0,0,0,0.07,0.04,0.06,0.06,365,-0.084615,0,0
2933,25.69,25.38,25.19,25.94,123,38,70,50,0.12,0.08,0.11,0.09,380,0.288462,0,0


In [154]:
train_output.head()

3585    1
6310    0
7260    0
1685    0
2933    1
Name: Room_Occupancy_Count, dtype: int32

# 3. Seguimiento de la huella de carbono

## 1. *Logistic Regression*:


In [156]:
#Scaling features data
#scaler = StandardScaler()
#scaled_train_features = scaler.fit_transform(train_features)

### CodeCarbon:

In [157]:
# Tracking training with codecarbon tool
tracker = EmissionsTracker()
lg_pipeline = Pipeline([("scaler", StandardScaler()), ("logistic_regression", LogisticRegression()),])

tracker.start()
lg_pipeline.fit(train_features, train_output)
emissions: float = tracker.stop()
print(f"Emissions: {emissions} kg")

[codecarbon INFO @ 17:20:39] [setup] RAM Tracking...
[codecarbon INFO @ 17:20:39] [setup] GPU Tracking...
[codecarbon INFO @ 17:20:39] Tracking Nvidia GPU via pynvml
[codecarbon INFO @ 17:20:39] [setup] CPU Tracking...
[codecarbon WARNING @ 17:20:39] No CPU tracking mode found. Falling back on CPU constant mode.
[codecarbon INFO @ 17:20:41] CPU Model on constant consumption mode: Intel(R) Core(TM) i5-9400F CPU @ 2.90GHz
[codecarbon INFO @ 17:20:41] >>> Tracker's metadata:
[codecarbon INFO @ 17:20:41]   Platform system: Windows-10-10.0.19043-SP0
[codecarbon INFO @ 17:20:41]   Python version: 3.10.4
[codecarbon INFO @ 17:20:41]   Available RAM : 7.935 GB
[codecarbon INFO @ 17:20:41]   CPU count: 6
[codecarbon INFO @ 17:20:41]   CPU model: Intel(R) Core(TM) i5-9400F CPU @ 2.90GHz
[codecarbon INFO @ 17:20:41]   GPU count: 1
[codecarbon INFO @ 17:20:41]   GPU model: 1 x GeForce GT 710
[codecarbon INFO @ 17:20:41] Energy consumed for RAM : 0.000000 kWh. RAM Power : 2.97562837600708 W
[codeca

Emissions: 1.1443766882514693e-07 kg


## 2. *Random Forest*

### CodeCarbon:

In [161]:
# Tracking training with codecarbon tool
rnd_clf = RandomForestClassifier(n_estimators=100, max_leaf_nodes=16, n_jobs=-1)
tracker = EmissionsTracker()

tracker.start()
rnd_clf.fit(train_features, train_output)
emissions: float = tracker.stop()
print(f"Emissions: {emissions} kg")

[codecarbon INFO @ 17:26:38] [setup] RAM Tracking...
[codecarbon INFO @ 17:26:38] [setup] GPU Tracking...
[codecarbon INFO @ 17:26:38] Tracking Nvidia GPU via pynvml
[codecarbon INFO @ 17:26:38] [setup] CPU Tracking...
[codecarbon WARNING @ 17:26:38] No CPU tracking mode found. Falling back on CPU constant mode.
[codecarbon INFO @ 17:26:40] CPU Model on constant consumption mode: Intel(R) Core(TM) i5-9400F CPU @ 2.90GHz
[codecarbon INFO @ 17:26:40] >>> Tracker's metadata:
[codecarbon INFO @ 17:26:40]   Platform system: Windows-10-10.0.19043-SP0
[codecarbon INFO @ 17:26:40]   Python version: 3.10.4
[codecarbon INFO @ 17:26:40]   Available RAM : 7.935 GB
[codecarbon INFO @ 17:26:40]   CPU count: 6
[codecarbon INFO @ 17:26:40]   CPU model: Intel(R) Core(TM) i5-9400F CPU @ 2.90GHz
[codecarbon INFO @ 17:26:40]   GPU count: 1
[codecarbon INFO @ 17:26:40]   GPU model: 1 x GeForce GT 710
[codecarbon INFO @ 17:26:41] Energy consumed for RAM : 0.000000 kWh. RAM Power : 2.97562837600708 W
[codeca

Emissions: 3.777838542553222e-07 kg


## 3. *Support Vector Machine* (SVM)

### CodeCarbon

In [160]:
# Tracking training with codecarbon tool
svm_pipeline = Pipeline([("scaler", StandardScaler()),("linear_svc", LinearSVC(C=1, loss="hinge")),])
tracker = EmissionsTracker()

tracker.start()
svm_pipeline.fit(train_features, train_output)
emissions: float = tracker.stop()
print(f"Emissions: {emissions} kg")

[codecarbon INFO @ 17:26:23] [setup] RAM Tracking...
[codecarbon INFO @ 17:26:23] [setup] GPU Tracking...
[codecarbon INFO @ 17:26:23] Tracking Nvidia GPU via pynvml
[codecarbon INFO @ 17:26:23] [setup] CPU Tracking...
[codecarbon WARNING @ 17:26:23] No CPU tracking mode found. Falling back on CPU constant mode.
[codecarbon INFO @ 17:26:25] CPU Model on constant consumption mode: Intel(R) Core(TM) i5-9400F CPU @ 2.90GHz
[codecarbon INFO @ 17:26:25] >>> Tracker's metadata:
[codecarbon INFO @ 17:26:25]   Platform system: Windows-10-10.0.19043-SP0
[codecarbon INFO @ 17:26:25]   Python version: 3.10.4
[codecarbon INFO @ 17:26:25]   Available RAM : 7.935 GB
[codecarbon INFO @ 17:26:25]   CPU count: 6
[codecarbon INFO @ 17:26:25]   CPU model: Intel(R) Core(TM) i5-9400F CPU @ 2.90GHz
[codecarbon INFO @ 17:26:25]   GPU count: 1
[codecarbon INFO @ 17:26:25]   GPU model: 1 x GeForce GT 710
C:\Users\INTEL I5\miniconda3\envs\TFG_Project\lib\site-packages\sklearn\svm\_base.py:1225: ConvergenceWarni

Emissions: 5.595417282859532e-08 kg
